In [1]:
import os,sys,trimesh
import numpy as np
sys.path.append('/home/anpei/Code/nglod/sdf-net')

os.environ['PYOPENGL_PLATFORM'] = 'egl'
os.environ['MESA_GL_VERSION_OVERRIDE'] = '3.3'
os.environ['MESA_GLSL_VERSION_OVERRIDE'] = '330'

import torch
from lib.torchgp import load_obj, point_sample, sample_surface, compute_sdf, normalize

In [2]:
def load_obj(path):
    mesh = trimesh.load(path)
    mesh.vertices = normalize(mesh.vertices)
    return mesh
    
def normalize(V):

    # Normalize mesh
    V_max = np.max(V, axis=0)
    V_min = np.min(V, axis=0)
    V_center = (V_max + V_min) / 2.
    V = V - V_center

    # Find the max distance to origin
    max_dist = np.sqrt(np.max(np.sum(V**2, axis=-1)))
    V_scale = 1. / max_dist
    V *= V_scale
    return V


def resample(V,F,num_samples, chunk=10000, sample_mode=['rand', 'near', 'near', 'near', 'near']):
    """Resample SDF samples."""

    points, sdfs = [],[]
    for _ in range(num_samples//chunk):

        pts = point_sample(V, F, sample_mode, chunk)
        sdf = compute_sdf(V, F, pts.cuda())   
        points.append(pts.cpu())
        sdfs.append(sdf.cpu())
    return torch.cat(points), torch.cat(sdfs)

In [3]:
for item in ['statuette.ply','dragon.ply','armadillo.obj','lucy.ply']:#'statuette.ply','dragon.ply','armadillo.obj'
    dataset_path = f'/home/anpei/Dataset/mesh/obj/{item}'
    mesh = load_obj(dataset_path)
    
    f = SDF(mesh.vertices, mesh.faces);

    V = torch.from_numpy(mesh.vertices).float().cuda()
    F = torch.from_numpy(mesh.faces).cuda()
    
    pts_train, _ = resample(V, F, num_samples=int(8*1024*1024/5))
    sdf_train = f(pts_train.numpy())
    
    pts_test, _ = resample(V, F, num_samples=int(16*1024*1024//5))
    sdf_test = f(pts_test.numpy())

    np.save(f'/home/anpei/Dataset/mesh/obj/sdf/{item[:-4]}_8M',{'points_train':pts_train.numpy(),'sdfs_train':sdf_train, \
                                                            'points_test':pts_test.numpy(),'sdfs_test':sdf_test})
